# King County Real Estate
---
Create a sortable dataframe based on calculated means, medians, etc. for the cities within King County.

In [1]:
import pandas as pd

# import sales data and clean date column
sales = pd.read_csv("kc_house_data.csv") 
sales['date'] = pd.to_datetime(sales['date']) 
# import zip code data and strips every state except WA
zips = pd.read_csv("uszips.csv")
zips_wa = zips[zips.state_id == 'WA']
pd.set_option('display.max_columns', None)
print('Overview of Sales Data:')
sales.head()

Overview of Sales Data:


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


## Functions
---

In [2]:
def zip_finder(code):
    """
    [Unused]
    Takes a zip code as input
    Returns the city
    """
    finder = zips_wa["zip"] == int(code)
    found = zips_wa[finder]
    return found["city"].values

def city_median(city_list):
    """
    Takes a list of city names as input
    Returns the city stats
    """
    stats = pd.DataFrame()
    for i in city_list:
        found = sales_concat[sales_concat['city'].str.match(str(i))]
        sqft = round((found['sqft_living'].astype(int).median()),2)
        price = round((found['price'].astype(int).median()),2)
        rooms = round((found['bedrooms'].astype(int).mean()),2)
        year = found['yr_built'].astype(int).median()
        acres = found['sqft_lot'].astype(int).mean()
        acreage = round((acres / 43560),2)
        baths = round((found['bathrooms'].astype(int).mean()),2)
        stats = stats.append({"Median Sqft":sqft,"Median Price":price,"Avg Rooms":rooms,
                              "Median Year":int(year),"Avg Acres":acreage,"Avg Baths":baths},ignore_index=True)
    stats["Median Year"] = pd.to_numeric(stats["Median Year"]).astype(int)
    stats["Median Price"] = pd.to_numeric(stats["Median Price"]).astype(int)
    stats["Median Sqft"] = pd.to_numeric(stats["Median Sqft"]).astype(int)
    return stats


def sorter(df, column, direction):
    """
    Takes the concat dataframe, column, and sort direction as input
    Sorts by column name
    Sorts by direction (ascending, descending)
    """
    print('Cities DataFrame sorted (' + str(direction)+ ') by ' + str(column))
    if direction == 'Descending':
        return df.sort_values(by=[str(column)], ascending=False)
    elif direction == 'Ascending':
        return df.sort_values(by=[str(column)], ascending=True)

## Data Prep
---

In [3]:
tester = sales['zipcode'].tolist()
city_by_zip = []
for i in tester: # creates list of corresponding city for each zip in row
    city_by_zip.append(zip_finder(i))

# concatonate the list with the original df, move the new column towards the front
city = pd.DataFrame(city_by_zip, columns=['city'])
sales_concat = pd.concat([sales, city], axis=1)
temp_city = sales_concat['city']
sales_concat.drop(labels=['city'], axis=1,inplace = True)
sales_concat.insert(2, 'city', temp_city)

# show only the cities with the total sales that occured in each
n = sales_concat['city'].value_counts()
cities = pd.DataFrame(n)
cities = cities.reset_index() # Fixes column/index issue
cities.columns = ['City', 'Sale Count']
cities['City'] = cities['City'].astype('str')

## Statistics
---

In [4]:
lst = cities['City'].tolist()
median_stats = city_median(lst) # feed list of cities into function
city_stats = pd.concat([cities,median_stats],axis=1)
city_stats = pd.DataFrame(city_stats, columns = ['City', 'Avg Rooms', 'Avg Baths', 'Median Sqft', 
                                                'Avg Acres', 'Median Price', 'Median Year', 'Sale Count'])
sorter(city_stats, "Avg Rooms", "Descending")

Cities DataFrame sorted (Descending) by Avg Rooms


,City,Avg Rooms,Avg Baths,Median Sqft,Avg Acres,Median Price,Median Year,Sale Count
23,Medina,4.06,2.84,3560,0.40,1892500,1966,50
14,Mercer Island,4.03,2.31,3020,0.31,993750,1968,282
2,Bellevue,3.82,1.98,2430,0.30,749000,1968,1407
7,Sammamish,3.73,2.17,2775,0.38,688500,1992,800
17,Bothell,3.55,1.85,2200,0.26,470000,1983,195
11,Woodinville,3.52,1.92,2460,0.94,570000,1983,471
12,Snoqualmie,3.52,2.09,2445,0.39,500000,2004,310
5,Kirkland,3.51,1.81,1980,0.23,510000,1976,977
1,Renton,3.51,1.74,1990,0.32,358000,1979,1597
9,Issaquah,3.49,2.11,2230,0.51,572000,1994,733


## Sources
---
Zips from: https://simplemaps.com/data/us-zips

Sales from: https://www.kaggle.com/harlfoxem/housesalesprediction